# Random Search for Hyperparameter Tuning

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import math
import zipfile
import urllib.request
import io
import warnings
from xgboost import XGBClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
warnings.filterwarnings('ignore')
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform,loguniform,randint

In [3]:
# raw data
url = "https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/ehr_preprocessed_seq_by_day_cat_embedding.pkl.zip"
with urllib.request.urlopen(url) as response:
    with zipfile.ZipFile(io.BytesIO(response.read())) as zip_file:
        with zip_file.open("ehr_preprocessed_seq_by_day_cat_embedding.pkl", "r") as file:
            EHR = pd.read_pickle(file)
train = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/train.csv")
valid = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/valid.csv")
test = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/test.csv")
print(train.shape)
print(test.shape)
print(valid.shape)
print(len(EHR['feature_cols']))

(55941, 13)
(17933, 12)
(13598, 13)
171


In [4]:
# combined data
# load processed data
train_mean = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_mean_mode.csv')
train_latest = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_latest.csv')
train_dispers = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_entropy_std.csv')
train_min = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_min.csv')
train_max = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_max.csv')
train_q1 = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_q1.csv')
train_q3 = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_q3.csv')
train_kurtosis = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_kurtosis.csv')
train_skewness = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_skewness.csv')
train_range = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_range.csv')
train_iqr = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/train/train_valid_iqr.csv')
train_latest = train_latest.drop(train_latest.columns[0:13], axis=1)
train_dispers = train_dispers.drop(train_dispers.columns[0:13], axis=1)
train_min = train_min.drop(train_min.columns[0:13], axis=1)
train_max = train_max.drop(train_max.columns[0:13], axis=1)
train_q1 = train_q1.drop(train_q1.columns[0:13], axis=1)
train_q3 = train_q3.drop(train_q3.columns[0:13], axis=1)
train_kurtois = train_kurtosis.drop(train_kurtosis.columns[0:13], axis=1)
train_skewness = train_skewness.drop(train_skewness.columns[0:13], axis=1)
train_range = train_range.drop(train_range.columns[0:13], axis=1)
train_iqr = train_iqr.drop(train_iqr.columns[0:13], axis=1)

# merge train_ and train_latest
train = pd.concat([train_mean, train_latest, train_dispers, train_min, train_max], axis=1)

# load processed data
test_mean = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_mean_mode.csv')
test_latest = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_latest.csv')
test_dispers = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_entropy_std.csv')
test_min = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_min.csv')
test_max = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_max.csv')
test_q1 = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_q1.csv')
test_q3 = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_q3.csv')
test_kurtosis = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_kurtosis.csv')
test_skewness = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_skewness.csv')
test_range = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_range.csv')
test_iqr = pd.read_csv('https://raw.githubusercontent.com/zxyao5148/STAT3612_2023_1A_GroupProject/main/data_processing/test/test_iqr.csv')
test_latest = test_latest.drop(test_latest.columns[0:12], axis=1)
test_dispers = test_dispers.drop(test_dispers.columns[0:12], axis=1)
test_min = test_min.drop(test_min.columns[0:12], axis=1)
test_max = test_max.drop(test_max.columns[0:12], axis=1)
test_q1 = test_q1.drop(test_q1.columns[0:12], axis=1)
test_q3 = test_q3.drop(test_q3.columns[0:12], axis=1)
test_kurtosis = test_kurtosis.drop(test_kurtosis.columns[0:12], axis=1)
test_skewness = test_skewness.drop(test_skewness.columns[0:12], axis=1)
test_range = test_range.drop(test_range.columns[0:12], axis=1)
test_iqr = test_iqr.drop(test_iqr.columns[0:12], axis=1)

# merge train_mean and train_latest
test = pd.concat([test_mean, test_latest, test_dispers, test_min, test_max], axis=1)

In [5]:
# separate response and features
X_train = train.drop(train.columns[0:13], axis=1)
y_train = train["readmitted_within_30days"]
X_test = test.drop(test.columns[0:12], axis=1)

In [6]:
# column categories
cat_cols = [EHR['feature_cols'][i] for i in EHR['cat_idxs']]
icd_cols = EHR['icd_cols']
cat_cols.extend(icd_cols)
num_cols = list(set(EHR['feature_cols']) - set(cat_cols)) 
print(cat_cols)
print(num_cols)

['gender', 'ethnicity', 'Creatinine Blood', 'Sodium Blood', 'pO2 Blood', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Lactate Blood', 'Anion Gap Blood', 'Eosinophils Joint Fluid', 'Hemoglobin Blood', 'Chloride Blood', 'Eosinophils Ascites', 'pH Urine', 'Calcium, Total Blood', 'Lymphocytes Ascites', 'Eosinophils Other Body Fluid', 'Eosinophils Blood', 'Lymphocytes Blood', 'Basophils Joint Fluid', 'Hematocrit Blood', 'Potassium Blood', 'H Blood', 'Monocytes Blood', 'Eosinophils Pleural', 'Troponin T Blood', 'Neutrophils Blood', 'Bicarbonate Blood', 'Basophils Blood', 'Glucose Blood', 'Basophils Ascites', 'pH Blood', 'Platelet Count Blood', 'Lymphocytes Other Body Fluid', 'Monocytes Ascites', 'Lymphocytes Joint Fluid', 'Lymphocytes Pleural', 'pCO2 Blood', 'Y90-Y99', 'G30-G32', 'O85-O92', 'C60-C63', 'F40-F48', 'M80-M85', 'R00-R09', 'J90-J94', 'A00-A09', 'E00-E07', 'F01-F09', 'F30-F39', 'H30-H36', 'D60-D64', 'N00-N08', 'F60-F69', 'I80-I89', 'I95-I99', 'N30-N39', 'K55-K64', 'F50-F59',

In [7]:
num_cols_latest = [f"{col}_latest" for col in num_cols]
num_cols_mean_mode = [f"{col}_mean_mode" for col in num_cols]
num_cols_entropy_std = [f"{col}_entropy_std" for col in num_cols]
num_cols_min = [f"{col}_min" for col in num_cols]
num_cols_max = [f"{col}_max" for col in num_cols]

In [ ]:
# LogisticRegression

logistic_model = LogisticRegression(solver='liblinear', max_iter=100)


param_grid = {
    'C': uniform(1e-6, 1e+6),  # Regularization parameter
    'penalty': ['l1', 'l2'],  # Regularization type
}


random_search = RandomizedSearchCV(logistic_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc',n_jobs=-1)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250

Best Parameters: {'C': 30538.762565708555, 'penalty': 'l2'}
Best Score: 0.7753466314968941


In [22]:
# Linear Discriminant Analysis
from sklearn.model_selection import RandomizedSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.stats import uniform

lda_model = LinearDiscriminantAnalysis(solver='lsqr')

param_grid = {
    'shrinkage': uniform(0, 1)  # Shrinkage parameter
}

random_search = RandomizedSearchCV(lda_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', n_jobs=-1)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'shrinkage': 0.0028570605198433574}
Best Score: 0.7790540758827025


In [24]:
# Quadratic Discriminant Analysis
qda_model = QuadraticDiscriminantAnalysis()

param_grid = {
    'reg_param': uniform(0, 1)  # Regularization parameter
}

random_search = RandomizedSearchCV(qda_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', n_jobs=-1)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  war

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  war

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  war

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  war

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  war

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  war

Best Parameters: {'reg_param': 0.027133939619383263}
Best Score: 0.7452109836592705


In QDA, there is no need to choose a solver method like in LDA because the covariance matrix is estimated independently for each class.

In [27]:
# Create an instance of the Gaussian Naive Bayes model
gnb_model = GaussianNB()

param_grid = {
    'var_smoothing': loguniform(1e-9, 1e-7)  # Smoothing parameter
}

random_search = RandomizedSearchCV(gnb_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', n_jobs=-1)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'var_smoothing': 9.826297914743841e-08}
Best Score: 0.6415869608530107


In [45]:
# KNeighborsClassifier
knn_model = KNeighborsClassifier()

param_grid = {
    'n_neighbors': randint(1, 100) # Number of neighbors
}

random_search = RandomizedSearchCV(knn_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', n_jobs=-1)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 181, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Use

  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'

  warnings.warn(
/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/

/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 181, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Use

  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_pairwise_distances_reduction/_dispatcher.py", line 259, in compute
    return ArgKmin64.compute(
           ^^^^^^^^^^^^^^^^^^
  File "sklearn/metrics/_pairwise_distances_reduction/_argkmin.pyx", line 90, in sklearn.metrics._pairwise_distances_reduction._argkmin.ArgKmin64.compute
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/fixes.py", line 74, in threadpool_limits
    return threadpoolctl.threadpool_limits(limits=limits, user_api=user_api)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 171, in __init__
    self._original_info = self._set_threadpool_limits()
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 268, in _set_threadpool_limits
    modules = _ThreadpoolInfo(pre

    y_pred = method_caller(clf, "predict_proba", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 194, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py", line 355, in predict_proba
    neigh_ind = self.kneighbors(X, return_distance=False)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_base.py", line 822, in kneighbors
    results = ArgKmin.compute(
              ^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python

...

    y_pred = method_caller(clf, "predict_proba", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 194, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py", line 355, in predict_proba
    neigh_ind = self.kneighbors(X, return_distance=False)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_base.py", line 822, in kneighbors
    results = ArgKmin.compute(
              ^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python

    y_pred = method_caller(clf, "predict_proba", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 194, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py", line 355, in predict_proba
    neigh_ind = self.kneighbors(X, return_distance=False)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_base.py", line 822, in kneighbors
    results = ArgKmin.compute(
              ^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python

  File "sklearn/metrics/_pairwise_distances_reduction/_argkmin.pyx", line 90, in sklearn.metrics._pairwise_distances_reduction._argkmin.ArgKmin64.compute
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/fixes.py", line 74, in threadpool_limits
    return threadpoolctl.threadpool_limits(limits=limits, user_api=user_api)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 171, in __init__
    self._original_info = self._set_threadpool_limits()
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 268, in _set_threadpool_limits
    modules = _ThreadpoolInfo(prefixes=self._prefixes,
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 340, in __init__
    self._load_modules()
 

           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 466, in _score
    y_pred = method_caller(clf, "predict_proba", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 194, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py", line 355, in predict_proba
    neigh_ind = self.kneighbors(X, return_distance=False)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yommiqvq/a

Best Parameters: {'n_neighbors': 52}
Best Score: nan


Increase the number of iterations (`n_iter`) and Adjust the parameter grid (ncreasing or decreasing the range of the `n_neighbors` parameter) still does not give a value of rocauc score

it is possible that the model is not able to converge or is encountering other issues during training

In [8]:
# Support Vector Machine

svm_model = SVC()
param_grid = {
    'C': uniform(0.1, 1e+6),  # Regularization parameter
    'degree': randint(1, 8),  # Degree of the polynomial kernel function
    'gamma': uniform(0.1, 1e+3)  # Kernel coefficient
}

random_search = RandomizedSearchCV(svm_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)


best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

In [52]:
# Classification Tree

tree_model = DecisionTreeClassifier()

param_grid = {
    'max_depth': randint(1, 100),  # Maximum depth of the tree
    'min_samples_split': randint(2, 100),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 100),  # Minimum number of samples required to be at a leaf node
}

random_search = RandomizedSearchCV(tree_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': 11, 'min_samples_leaf': 81, 'min_samples_split': 9}
Best Score: 0.7523838486569094


In [11]:
bagging_model = BaggingClassifier()

param_grid = {
    'n_estimators': randint(10, 100),  # Number of base estimators
    'max_samples': uniform(0.1, 1.0),  # Number of samples to draw from X to train each base estimator
    'max_features': uniform(0.1, 1.0),  # Number of features to draw from X to train each base estimator
}

random_search = RandomizedSearchCV(bagging_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_features': 0.42654076880583536, 'max_samples': 0.6704439744053994, 'n_estimators': 71}
Best Score: 0.7891346006992203


In [8]:
# Random Forest
rf_model = RandomForestClassifier()

param_grid = {
    'n_estimators': randint(10, 100),  # Number of trees in the forest
    'max_depth': randint(1, 100),  # Maximum depth of the tree
    'min_samples_split': randint(2, 100),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 100),  # Minimum number of samples required to be at a leaf node
}

random_search = RandomizedSearchCV(rf_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': 60, 'min_samples_leaf': 14, 'min_samples_split': 10, 'n_estimators': 99}
Best Score: 0.7937354798619498


In [59]:
# XGBoost model
xgb_model = XGBClassifier()

param_grid = {
    'n_estimators': randint(10,100),  # Number of boosting rounds
    'max_depth': randint(1, 100),  # Maximum depth of a tree
    'learning_rate': uniform(1e-6, 1e+1),  # Learning rate
    'subsample': uniform(0.1, 1),  # Subsample ratio of the training instances
    'colsample_bytree': uniform(0.1, 1),  # Subsample ratio of columns when constructing each tree
    #'gamma': uniform(0, 10),  # Minimum loss reduction required to make a further partition on a leaf node
    #'reg_alpha': uniform(0, 10),  # L1 regularization term on weights
    #'reg_lambda': uniform(0, 10),  # L2 regularization term on weights
    'min_child_weight': randint(1, 100),  # Minimum sum of instance weight needed in a child,
    'scale_pos_weight': uniform(1, 10)
}

random_search = RandomizedSearchCV(xgb_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'colsample_bytree': 0.550499251969543, 'learning_rate': 0.13265061159866529, 'max_depth': 89, 'min_child_weight': 60, 'n_estimators': 23, 'scale_pos_weight': 9.08397348116461, 'subsample': 0.40461376917337066}
Best Score: 0.7885494219538076


In [10]:
# Gradient Boosting model
gb_model = GradientBoostingClassifier()

param_grid = {
    'n_estimators': randint(10, 100),  # Number of boosting stages
    'learning_rate': uniform(1e-6, 1e+1),  # Learning rate
    'max_depth': randint(1, 100),  # Maximum depth of each tree
    'min_samples_split': randint(2, 100),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 100)  # Minimum number of samples required to be at a leaf node
    #'max_features': ['auto', 'sqrt', 'log2', None],  # Number of features to consider when looking for the best split
    #'subsample': uniform(0.1, 0.9)  # Subsample ratio of the training instances
}

random_search = RandomizedSearchCV(gb_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
best_score = random_search.best_score_ 

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'learning_rate': 0.07066405219717406, 'max_depth': 89, 'min_samples_leaf': 49, 'min_samples_split': 92, 'n_estimators': 68}
Best Score: 0.7951183165352813


In [60]:
# AdaBoost model
ada_model = AdaBoostClassifier()

param_grid = {
    'n_estimators': randint(10, 100),  # Number of estimators
    'learning_rate': uniform(1e-6, 1e+1),  # Learning rate
    #'algorithm': ['SAMME', 'SAMME.R'],  # Algorithm for boosting
    #'base_estimator__max_depth': randint(1, 20),  # Maximum depth of each base estimator
    #'base_estimator__min_samples_split': randint(2, 10),  # Minimum number of samples required to split an internal node
    #'base_estimator__min_samples_leaf': randint(1, 10),  # Minimum number of samples required to be at a leaf node
    #'base_estimator__max_features': ['auto', 'sqrt', 'log2', None]  # Number of features to consider when looking for the best split

}
random_search = RandomizedSearchCV(ada_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'learning_rate': 0.34388621115218393, 'n_estimators': 87}
Best Score: 0.7856919662048996
